In [66]:
import tensorflow as tf
import tensorflow
import tensorflow_text as text
import tensorflow_hub as hub

shallow_mlp_model = tf.keras.models.load_model("./Models/workPlzModel1model3")

bert_preprocess = hub.KerasLayer("./Models/bert_en_uncased_preprocess_3")
bert_encoder = hub.KerasLayer("./Models/bert_en_uncased_L-12_H-768_A-12_4")

OSError: ./Models/bert_en_uncased_preprocess_3 does not exist.

In [ ]:
from spacy.lang.en import English
from spacy.lang.en.punctuation import ALPHA
from spacy.lang.en.stop_words import STOP_WORDS
import re

model_for_inference = shallow_mlp_model
nlp = English()
def cleanPlot(qq):
    tokenizer = nlp.tokenizer(qq.lower())
    tokenizer = " ".join([str(x) if str(x) not in STOP_WORDS else "" for x in tokenizer])
    punctuations = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''
    no_punc = ""
    for char in tokenizer:
        if char not in punctuations:
            no_punc = no_punc + char
    WHITE_SPACE_PATTERN = re.compile(r' +')
    return re.sub(WHITE_SPACE_PATTERN, ' ', no_punc.strip())

def test(testText):
    #encode = {'Action': 0, 'Fantasy': 1, 'Adventure': 2, 'Kids': 3, 'Drama': 4, 'Sci-Fi': 5, 'Music': 6, 'Shounen': 7, 'Slice of Life': 8, 'Romance': 9} #hardcoded for simplicity. Anyway this model predicts probability only for 10 tags
    encode = {'comedy': 0, 'action': 1, 'romance': 2, 'horror': 3, 'thriller': 4, 'western': 5, 'crime': 6, 'adventure': 7, 'musical': 8, 'family': 9}
    print(f"Synopsis: {testText}")
    inference_dataset = tf.data.Dataset.from_tensors(cleanPlot(testText)).batch(1)
    text_batch = next(iter(inference_dataset))
    predicted_probabilities = model_for_inference.predict(text_batch)

    for i, text in enumerate(text_batch[:5]):
        predicted_proba = [proba for proba in predicted_probabilities[i]]
        top_labels = [(p,x) for p, x in sorted(
            zip(predicted_proba, list(encode.keys())),
            key=lambda pair: pair[0],
            reverse=True,
        )
                      ]
        print(f"Top 3 predicted Labels:\n ({', '.join([f'Probability of {label[1]}: {label[0]} ' for label in top_labels[:3]])})")
        print(" ")

def embed(smth):
    return bert_encoder(bert_preprocess(smth))['pooled_output']

In [ ]:
t = """Washington Delaware Jones (Mantan Moreland) has never done much good for his town, and ultimately he is ordered by a judge to leave for good. In following the judge's order he brings his friend and collaborator Jefferson (F.E. Miller), and the two men go on a search for a new place to live.\r\nBoth lack professional experience and start thinking about what kind of jobs they might be apt for. Since they both agree on liking food, they decide on becoming food tasters. When they come to the first destination on the road, they pretend to be food inspectors and start stealing chickens from a farm, but the farmer (Nathan Curry) shoots after them. They meet a man of some wealth named Brown (Harold Garrison), whose car has stopped alongside the road, and his friend Dawson (Jessie Cryer).\r\nWhile Brown's chauffeur (Napoleon Whiting) runs along to find gas, the four remaining men start throwing dice. Washington and Jefferson win all the other two have, including the car, and they are driven by the chauffeur to a nearby country club run by Dr. Brutus Blake (Maceo Bruce Sheffield). Blake is a swindler, and when he sees the two men arrive in their elegant car, he decides to take them for what they have. Blake arranges a crap game where outcome is fixed to his advantage. Since Blake's partner Blackstone (Arthur Ray) doesn't approve of his tactics, they argue, and Blackstone threaten to reveal to the guests what Blake is up to.\r\nBlake has a thing for the club hostess (Florence O'Brien), and later that night he sees Washington dance with her. He becomes jealous and challenges Washington to a fight. Blake manages to knock himself out during the fight, and when he wakes up he is more determined than ever to take the two guests money.\r\nEveryone is unaware that the place is haunted by Blake's dead relatives, and they are quite disappointed with how Blake has turned out. They also regret leaving the place to him in their wills and send one of them, uncle Ezra Dewey, to set Blake straight.\r\nThe gambling begins, and soon Washington and Jefferson has won the whole club from Blake through a bet. Ezra finds that the place is just as sinful and decadent under the management of Washington and Jefferson as it was under Blake. The former owners start scheming to get the place back, and deciding to let the local sheriff arrest them for made-up criminal offenses.\r\nEzra finds out about Blake's wicked plans and scares him off. All the ghosts then go on to scare off Washington and Jefferson, by haunting the club, and the two men flee for their lives.
"""
test(t)

In [62]:
test("""At the renowned Shuchiin Academy, Miyuki Shirogane and Kaguya Shinomiya are the student body's top representatives. Ranked the top student in the nation and respected by peers and mentors alike, Miyuki serves as the student council president. Alongside him, the vice president Kaguya—eldest daughter of the wealthy Shinomiya family—excels in every field imaginable. They are the envy of the entire student body, regarded as the perfect couple.

However, despite both having already developed feelings for the other, neither are willing to admit them. The first to confess loses, will be looked down upon, and will be considered the lesser. With their honor and pride at stake, Miyuki and Kaguya are both equally determined to be the one to emerge victorious on the battlefield of love!""")

Synopsis: At the renowned Shuchiin Academy, Miyuki Shirogane and Kaguya Shinomiya are the student body's top representatives. Ranked the top student in the nation and respected by peers and mentors alike, Miyuki serves as the student council president. Alongside him, the vice president Kaguya—eldest daughter of the wealthy Shinomiya family—excels in every field imaginable. They are the envy of the entire student body, regarded as the perfect couple.

However, despite both having already developed feelings for the other, neither are willing to admit them. The first to confess loses, will be looked down upon, and will be considered the lesser. With their honor and pride at stake, Miyuki and Kaguya are both equally determined to be the one to emerge victorious on the battlefield of love!
Top 3 predicted Labels:
 (Probability of adventure: 0.3455441892147064 , Probability of romance: 0.3001581132411957 , Probability of comedy: 0.20585787296295166 )
 


In [64]:
test("""The arrival of nurse Ellen Burton to the Belgian Congo is unwelcome to hunter John "Lonni" Douglas (Robert Mitchum), who captures animals for zoos. He warns her against traveling upriver to join a female doctor who is working with native tribesmen.\r\nShort of money, Lonni is intrigued when partner Huysman (Walter Slezak) tells him there is gold to be found in the region where Ellen will be traveling. Lonni volunteers to accompany her, along with gun bearer Jacques.\r\nEllen (Susan Hayward) is a widow who once discouraged her physician husband from his dream of coming to Africa to give medical aid. She talks a witch doctor out of killing a woman with an abscessed tooth. Upset with her, the witch doctor places a deadly tarantula in Ellen\'s tent.\r\nThe doctor she is there to assist has died of fever. The king is pleased when his son is saved from a lion by Lonni, his wounds treated by Ellen, but then the king takes her hostage when Huysman, heavily armed, arrives to search for gold. Huysman\'s men knock Lonni unconscious and tie him up, but Jacques sacrifices his own life to save that of Lonni, who returns to Ellen\'s side for good""")

Synopsis: The arrival of nurse Ellen Burton to the Belgian Congo is unwelcome to hunter John "Lonni" Douglas (Robert Mitchum), who captures animals for zoos. He warns her against traveling upriver to join a female doctor who is working with native tribesmen.
Short of money, Lonni is intrigued when partner Huysman (Walter Slezak) tells him there is gold to be found in the region where Ellen will be traveling. Lonni volunteers to accompany her, along with gun bearer Jacques.
Ellen (Susan Hayward) is a widow who once discouraged her physician husband from his dream of coming to Africa to give medical aid. She talks a witch doctor out of killing a woman with an abscessed tooth. Upset with her, the witch doctor places a deadly tarantula in Ellen's tent.
The doctor she is there to assist has died of fever. The king is pleased when his son is saved from a lion by Lonni, his wounds treated by Ellen, but then the king takes her hostage when Huysman, heavily armed, arrives to search for gold. Hu

In [65]:
test("""John tries to defeat mafia and learns to use a gun to protect himself. But only pistol is not a way, so hhe should try to find comrades in the mafia.
""")

Synopsis: John tries to defeat mafia and learns to use a gun to protect himself. But only pistol is not a way, so hhe should try to find comrades in the mafia.

Top 3 predicted Labels:
 (Probability of romance: 0.2765181064605713 , Probability of comedy: 0.22085535526275635 , Probability of horror: 0.20856410264968872 )
 
